In [2]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "my_database", "my_table")
(
    storage.schema_builder()
    .add_field("_id", DataType.INT)
    .add_field("content", DataType.STRING, [FieldOption.ANALYZE])
    .add_array_field("embedding", DataType.FLOAT)
    .execute()
)

# 构建索引
data = [
    {"_id": "1", "content": "Hello world", "embedding": [0.1, 0.2, 0.3]},
    {"_id": "2", "content": "OpenAI GPT", "embedding": [0.4, 0.5, 0.6]}
]

storage.write_builder().add_items(data).execute()

storage.commit()

# 搜索
results = storage.query_builder().set_vector([0.1, 0.2, 0.3], "embedding").set_keyword("Hello").execute()

print(results)

2024-07-05 23:37:46.710 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-07-05 23:37:46,916	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-07-05 23:37:46,917	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.
2024-07-05 23:37:46,936	WARNING __init__.py:158 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.


21


CrossLanguageError: An exception raised from JAVA:
io.ray.api.exception.RayTaskException: (pid=59909, ip=192.168.3.13) Error executing task 86755313f81a7117b0da1ab57a026afaa938746412000000
	at io.ray.runtime.task.TaskExecutor.execute(TaskExecutor.java:186)
	at io.ray.runtime.RayNativeRuntime.nativeRunTaskExecutor(Native Method)
	at io.ray.runtime.RayNativeRuntime.run(RayNativeRuntime.java:231)
	at io.ray.runtime.runner.worker.DefaultWorker.main(DefaultWorker.java:15)
Caused by: io.ray.api.exception.RayTaskException: (pid=59982, ip=192.168.3.13) Error executing task a0166564385ffb96be295d1aa077ea5f0dab7cbc12000000
	... 4 more
Caused by: java.lang.RuntimeException: not support
	at tech.mlsql.retrieval.schema.SimpleSchemaParser._parse(SimpleSchemaParser.java:88)
	at tech.mlsql.retrieval.schema.SimpleSchemaParser._parse(SimpleSchemaParser.java:84)
	at tech.mlsql.retrieval.schema.SimpleSchemaParser._parse(SimpleSchemaParser.java:67)
	at tech.mlsql.retrieval.schema.SimpleSchemaParser.parse(S

In [5]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "my_database", "my_table")
v = storage.retrieval.get_table_settings("byzerai_store","byzerai_store","text_content_chunk")
print(v)

2024-07-05 23:51:24.515 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-07-05 23:51:24,719	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-07-05 23:51:24,720	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


21
None
